In [1]:
from utils.datasets import AlphabetSortingDataset, NumberSortingDataset
from models.pointer_net import PointerNet
from torch.utils.data import DataLoader

In [2]:
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np

In [3]:
import warnings
warnings.filterwarnings("ignore")

params = {
    # Data
    'batch_size': 256,
    'shuffle': True,
    'nof_workers': 0, # must stay at 0
    #Train
    'nof_epoch': 1000,
    'lr': 0.0001,
    # GPU
    'gpu': True,
    # Network
    'embedding_size': 300,
    'hiddens': 512,
    'nof_lstms': 8,
    'dropout': 0.3,
    'bidir': False # True not working right now
}

dataset = AlphabetSortingDataset(300000, min_len=4, max_len=4)
dataloader = DataLoader(dataset,
                        batch_size=params['batch_size'],
                        shuffle=params['shuffle'])

In [4]:
if params['gpu'] and torch.cuda.is_available():
    USE_CUDA = True
    print('Using GPU, %i devices.' % torch.cuda.device_count())
else:
    USE_CUDA = False

Using GPU, 1 devices.


In [5]:
model = PointerNet(params['embedding_size'],
                   params['hiddens'],
                   params['nof_lstms'],
                   params['dropout'],
                   params['bidir'])

if USE_CUDA:
    model.cuda()
    net = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

CCE = torch.nn.CrossEntropyLoss()
model_optim = optim.Adam(filter(lambda p: p.requires_grad,
                                model.parameters()),
                                 lr=params['lr'])


In [6]:
from tqdm import tqdm
losses = []

for i_epoch, epoch in enumerate(range(params['nof_epoch'])):
    batch_loss = []
    iterator = tqdm(dataloader, unit='Batch')
    
    if i_epoch % 10 == 0 and i_epoch != 0:
        for param in model_optim.param_groups:
            param['lr'] *= 0.9
    
    for i_batch, sample_batched in enumerate(iterator):
        iterator.set_description('Epoch %i/%i' % (epoch+1, params['nof_epoch']))

        x, y, chars = sample_batched
        train_batch = Variable(x)
        target_batch = Variable(y)

        if USE_CUDA:
            train_batch = train_batch.cuda()
            target_batch = target_batch.cuda()

        o, p = model(train_batch)
        o = o.contiguous().view(-1, o.size()[-1])
        target_batch = target_batch.view(-1)
        
        loss = CCE(o, target_batch) #/ target_batch.shape[1] # need to take the length of the table into account
        #acc = get_accuracy(p, target_batch)
        
        losses.append(loss.data)
        batch_loss.append(loss.data)

        model_optim.zero_grad()
        loss.backward()
        model_optim.step()
        
        iterator.set_postfix(loss='{}'.format(loss.data))
    batch_loss = torch.Tensor(batch_loss)
    iterator.set_postfix(loss=np.average(batch_loss))

Epoch 1/1000:  70%|█████████████████████████▊           | 817/1172 [01:12<00:31, 11.34Batch/s, loss=1.1381863355636597]

KeyboardInterrupt: 

In [ ]:
model.eval()

In [ ]:
x, y, z = dataset[:100]
x = x.cuda()
y = y.cuda()

In [ ]:
o, p = model(x)

In [ ]:
sorting = list(zip(z, p.data.cpu().tolist()))

In [ ]:
sorting

In [ ]:
for pair in sorting:
    sequence, pointers = pair
    tmp = {}
    for seq, p in zip(sequence, pointers):
        tmp[seq] = p
    tmp =  [k for k, v in sorted(tmp.items(), key=lambda item: item[1])]
    print(tmp, " - ", sorted(sequence))